In [25]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import features
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
df = pd.read_csv('all_data_combined.csv')
df.head()

,subject_id,age,urea_n_min,urea_n_max,urea_n_mean,resprate_min,resprate_max,resprate_mean,glucose_min,glucose_max,...,oasis,lods,gender_F,gender_M,marital_status_DIVORCED,marital_status_MARRIED,marital_status_SEPARATED,marital_status_SINGLE,marital_status_WIDOWED,death
0,15057,58.831224,16.0,37.0,26.111111,10.0,52,20.104478,67.0,405.0,...,36.5,5.5,0,1,0,1,0,0,0,1
1,79262,81.618606,10.0,15.0,13.500000,11.0,31,18.704225,119.0,348.0,...,48.0,4.0,0,1,0,1,0,0,0,1
2,77191,79.102744,16.0,56.0,35.500000,20.0,31,25.744681,90.0,188.0,...,35.0,6.0,1,0,1,0,0,0,0,0
3,84966,88.232043,8.0,27.0,17.692308,10.0,51,17.770833,92.0,271.0,...,40.5,4.0,1,0,0,0,0,1,0,1
4,94997,90.000000,38.0,48.0,42.250000,4.0,40,26.360000,62.0,135.0,...,35.0,5.0,1,0,0,0,0,1,0,0


In [3]:
df.shape

(2152, 168)

In [51]:

model = RandomForestClassifier()
model.fit(df.iloc[0:1800,1:df.shape[1]-1], df.iloc[0:1800,df.shape[1]-1]) 
model.score(df.iloc[1800:,1:df.shape[1]-1], df.iloc[1800:,df.shape[1]-1])


0.7613636363636364

In [49]:
features=df.columns[1:len(df.columns)-1]
def getFeatures(weights):
    df_w= pd.DataFrame(features)
    df_w.columns =['features']
    df_w['weight']=weights
    return df_w

In [57]:
df_weight = getFeatures(model.feature_importances_)
df_weight.sort_values(by='weight',inplace=True,ascending=False)
df_weight

df_weight.to_csv("randomForest.csv",index = False)

In [52]:
def randomForest(df):
    
    kf = KFold(n_splits=3, random_state=0)
    
    result = []
    
    for train, test in kf.split(df):

        train_data = df.iloc[train,:]
        test_data =  df.iloc[test,:]

        trainx = train_data.iloc[:,1:(train_data.shape[1]-2)]
        trainy = train_data.iloc[:,train_data.shape[1]-1]

        testx = test_data.iloc[:,1:(test_data.shape[1]-2)]
        testy = test_data.iloc[:,test_data.shape[1]-1]

        model = RandomForestClassifier()
        model.fit(trainx, trainy)
        result.append(model.score(testx, testy))
               
    return np.average(result)

In [53]:
result=randomForest(df)
print(result)

0.7499957912948437


,features,weight
151,51277_var,0.044364
147,51250_var,0.030753
86,51256_max,0.028929
105,50902_min,0.025882
116,51244_min,0.022142
90,51277_max,0.021541
111,50983_min,0.019716
89,51275_max,0.019244
145,51248_var,0.018813
68,50882_max,0.017472


In [22]:
features=df.columns[1:len(df.columns)-1]

In [23]:
len(features)

166

In [24]:
features

Index(['age', 'urea_n_min', 'urea_n_max', 'urea_n_mean', 'resprate_min',
       'resprate_max', 'resprate_mean', 'glucose_min', 'glucose_max',
       'glucose_mean',
       ...
       'apsiii', 'oasis', 'lods', 'gender_F', 'gender_M',
       'marital_status_DIVORCED', 'marital_status_MARRIED',
       'marital_status_SEPARATED', 'marital_status_SINGLE',
       'marital_status_WIDOWED'],
      dtype='object', length=166)